In [15]:
!git clone https://github.com/waghts95/pyspark_pydeequ.git

Cloning into 'pyspark_pydeequ'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (6/6), 6.57 MiB | 19.32 MiB/s, done.


In [15]:
!pip3 install py4j==0.10.9.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.2
    Uninstalling py4j-0.10.9.2:
      Successfully uninstalled py4j-0.10.9.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyspark 3.2.0 requires py4j==0.10.9.2, but you have py4j 0.10.9.7 which is incompatible.


In [2]:
! pip3 install pyspark==3.3

  Using cached pyspark-3.3.0-py2.py3-none-any.whl
  Using cached py4j-0.10.9.5-py2.py3-none-any.whl (199 kB)
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.2
    Uninstalling py4j-0.10.9.2:
      Successfully uninstalled py4j-0.10.9.2
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.2.0
    Uninstalling pyspark-3.2.0:
      Successfully uninstalled pyspark-3.2.0


In [3]:
!pip3 install pydeequ==1.1.0

In [4]:
import os
os.environ["SPARK_VERSION"]="3.3"

In [5]:
from pyspark.sql import SparkSession

In [6]:
import pydeequ

In [7]:
spark = (SparkSession
    .builder
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .getOrCreate())


In [16]:
df = spark.read.csv('/content/pyspark_pydeequ/USvideos.csv', header=True)

In [17]:
df.select("*").show()

+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|        publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|5iVxBDPlbc4|     17.22.12|Why Imperial Admi...|     The Lore Master|         24|2017-12-18T18:19:...|"Why Imperial Adm...| 126622|  5601|      72|          535|https://i.ytimg.c...|            False|           Fal



```
# Analyzers
```



In [9]:
from pydeequ.analyzers import *

analysisResult = AnalysisRunner(spark) \
                    .onData(df) \
                    .addAnalyzer(Size()) \
                    .addAnalyzer(Completeness("b")) \
                    .run()

analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult)
analysisResult_df.show()

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+-------+--------+----+-------+
| entity|instance|name|  value|
+-------+--------+----+-------+
|Dataset|       *|Size|48137.0|
+-------+--------+----+-------+





```
# Profile
```



In [10]:
from pydeequ.profiles import *

result = ColumnProfilerRunner(spark) \
    .onData(df) \
    .run()

for col, profile in result.profiles.items():
    print(profile)

StandardProfiles for column: video_id: {
    "completeness": 1.0,
    "approximateNumDistinctValues": 7216,
    "dataType": "String",
    "isDataTypeInferred": false,
    "typeCounts": {
        "Boolean": 0,
        "Fractional": 0,
        "Integral": 0,
        "Unknown": 0,
        "String": 48137
    },
    "histogram": null
}
StandardProfiles for column: ratings_disabled: {
    "completeness": 0.85090470947504,
    "approximateNumDistinctValues": 4,
    "dataType": "String",
    "isDataTypeInferred": false,
    "typeCounts": {
        "Boolean": 0,
        "Fractional": 0,
        "Integral": 0,
        "Unknown": 7177,
        "String": 40960
    },
    "histogram": [
        [
            "NullValue",
            7177,
            0.14909529052496
        ],
        [
            " garganelli",
            7,
            0.00014541828531067577
        ],
        [
            "True",
            169,
            0.0035108128882148865
        ],
        [
            "False",
  



```
# Constraint Suggestion
```



In [11]:
from pydeequ.suggestions import *

suggestionResult = ConstraintSuggestionRunner(spark) \
             .onData(df) \
             .addConstraintRule(DEFAULT()) \
             .run()

# Constraint Suggestions in JSON format
print(suggestionResult)

{'constraint_suggestions': [{'constraint_name': 'CompletenessConstraint(Completeness(video_id,None))', 'column_name': 'video_id', 'current_value': 'Completeness: 1.0', 'description': "'video_id' is not null", 'suggesting_rule': 'CompleteIfCompleteRule()', 'rule_description': 'If a column is complete in the sample, we suggest a NOT NULL constraint', 'code_for_constraint': '.isComplete("video_id")'}, {'constraint_name': "ComplianceConstraint(Compliance('ratings_disabled' has value range 'False', 'True', ' garganelli', ' Ralph and Rickey’s',`ratings_disabled` IN ('False', 'True', ' garganelli', ' Ralph and Rickey’s'),None))", 'column_name': 'ratings_disabled', 'current_value': 'Compliance: 1', 'description': "'ratings_disabled' has value range 'False', 'True', ' garganelli', ' Ralph and Rickey’s'", 'suggesting_rule': 'CategoricalRangeRule(com.amazon.deequ.suggestions.rules.CategoricalRangeRule$$$Lambda$3730/0x000000084162f040@1f3df85d)', 'rule_description': 'If we see a categorical range 



```
# Constraint verification
```



In [12]:
from pydeequ.checks import *
from pydeequ.verification import *

check = Check(spark, CheckLevel.Warning, "Review Check")

checkResult = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(
        check.hasSize(lambda x: x >= 3) \
        .hasMin("b", lambda x: x == 0) \
        .isComplete("c")  \
        .isUnique("a")  \
        .isContainedIn("a", ["foo", "bar", "baz"]) \
        .isNonNegative("b")) \
    .run()

checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show()

Python Callback server started!
+------------+-----------+------------+--------------------+-----------------+--------------------+
|       check|check_level|check_status|          constraint|constraint_status|  constraint_message|
+------------+-----------+------------+--------------------+-----------------+--------------------+
|Review Check|    Warning|     Warning|SizeConstraint(Si...|          Failure|org.apache.spark....|
|Review Check|    Warning|     Warning|MinimumConstraint...|          Failure|Input data does n...|
|Review Check|    Warning|     Warning|CompletenessConst...|          Failure|Input data does n...|
|Review Check|    Warning|     Warning|UniquenessConstra...|          Failure|Input data does n...|
|Review Check|    Warning|     Warning|ComplianceConstra...|          Failure|org.apache.spark....|
|Review Check|    Warning|     Warning|ComplianceConstra...|          Failure|org.apache.spark....|
+------------+-----------+------------+--------------------+--------



```
# Repository
Save to a Metrics Repository by adding the useRepository() and saveOrAppendResult() calls to your Analysis Runner
```



In [13]:
from pydeequ.repository import *
from pydeequ.analyzers import *

metrics_file = FileSystemMetricsRepository.helper_metrics_file(spark, 'metrics.json')
repository = FileSystemMetricsRepository(spark, metrics_file)
key_tags = {'tag': 'pydeequ hello world'}
resultKey = ResultKey(spark, ResultKey.current_milli_time(), key_tags)

analysisResult = AnalysisRunner(spark) \
    .onData(df) \
    .addAnalyzer(ApproxCountDistinct('b')) \
    .useRepository(repository) \
    .saveOrAppendResult(resultKey) \
    .run()



```
# To load previous runs, use the repository object to load previous results back in.
```



In [14]:
result_metrep_df = repository.load() \
    .before(ResultKey.current_milli_time()) \
    .forAnalyzers([ApproxCountDistinct('b')]) \
    .getSuccessMetricsAsDataFrame()